In [1]:
import pandas as pd
import numpy as np

In [2]:
boolean_matrix = pd.read_csv('tf.csv')
boolean_matrix.head()

,name,1818,taking,epistolary,forgiv,lov,hum,what,then,august,...,growing,wal,goes,winds,releaseslistsexplorenews,comt,conferenc,com,months,project
0,http://quotes.toscrape.com/,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,http://quotes.toscrape.com/tag/inspirational/,0,0,0,0,1,0,1,1,0,...,0,0,1,0,0,0,0,1,0,0
2,http://quotes.toscrape.com/tag/paraphrased/pag...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,http://quotes.toscrape.com/tag/obvious/page/1/,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,http://quotes.toscrape.com/tag/misattributed-e...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# get_pj_qj

In [3]:
def get_pj_qj(boolean_matrix: pd.DataFrame)-> tuple:
    row_size = boolean_matrix.shape[1]
    pj = np.full((row_size), 0.5)
    qj = np.array(boolean_matrix.sum(axis=0) / row_size)
    return pj, qj

In [38]:
values = boolean_matrix.iloc[:,1:] # tomar solo las columnas con numero, ignorar la url
pj, qj = get_pj_qj(values)
print(pj)
print(qj)

[0.5 0.5 0.5 ... 0.5 0.5 0.5]
[0.00054795 0.00054795 0.00054795 ... 0.00438356 0.00054795 0.00054795]


# Consulta

In [22]:
import process_text
import re

In [23]:
consulta_raw = 'In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining the photoelectric effect, the basis of electronics, earned him the Nobel Prize in 1921. His first paper on Special Relativity Theory, also published in 1905, changed the world.'

In [24]:
corpus_stems = boolean_matrix.columns[1:]

In [12]:
def vector_consulta(consulta, corpus_stems):
    import re
    import process_text
    tokens = re.split(r"[^a-z0-9]+", consulta)
    query_data = process_text.ProcessData('query', tokens)
    vector_query = []
    for stem in corpus_stems:
        if stem in query_data.stems:
            vector_query.append(1)
        else:
            vector_query.append(0)
    return np.array(vector_query)

In [26]:
q = vector_consulta(consulta_raw, corpus_stems)

# semejanza

In [5]:
def get_sem_table(consulta_vector, boolean_matrix):
    q = consulta_vector
    pj, qj = get_pj_qj(boolean_matrix)
    semejanza = ( (q * boolean_matrix) *  np.log10((  pj*(1-qj)  )/ (  qj*(1-pj)  )) ).sum(axis=1)
    return semejanza

In [28]:
semejanza = get_sem_table(q, np.array(values))
semejanza

array([ 4.13081764,  4.0927896 ,  0.        ,  0.        ,  0.        ,
        0.        ,  8.66629914,  8.66629914,  0.        ,  0.        ,
        0.        ,  8.66629914,  0.        ,  6.57471132,  0.        ,
        5.08830045,  1.91357305,  4.0927896 ,  0.        ,  6.57471132,
        0.        ,  4.571823  ,  4.21844638,  0.        ,  4.0927896 ,
        2.17921656,  0.        ,  0.        ,  2.17921656,  0.        ,
        0.        ,  0.        ,  0.        ,  4.0927896 ,  0.        ,
        4.0927896 , 10.58065162,  0.        ,  0.        ,  0.        ,
        4.13081764,  4.13081764,  4.13081764,  4.13081764,  0.        ,
        7.74655041,  4.13081764, 11.41217783, 51.6815864 ,  6.43569098])

# Concatenar nombre y ordenar

In [29]:
resultado = boolean_matrix[['name']].copy()
resultado['semejanza'] = semejanza
resultado.head()

,name,semejanza
0,http://quotes.toscrape.com/,4.130818
1,http://quotes.toscrape.com/tag/inspirational/,4.092790
2,http://quotes.toscrape.com/tag/paraphrased/pag...,0.000000
3,http://quotes.toscrape.com/tag/obvious/page/1/,0.000000
4,http://quotes.toscrape.com/tag/misattributed-e...,0.000000


In [30]:
ordenados = resultado.sort_values(by=['semejanza'], ascending=False)
ordenados.head()

,name,semejanza
48,http://quotes.toscrape.com/author/Albert-Einst...,51.681586
47,http://quotes.toscrape.com/author/J-K-Rowling/,11.412178
36,http://quotes.toscrape.com/author/Marilyn-Monroe/,10.580652
11,http://quotes.toscrape.com/tag/love/page/1/,8.666299
6,http://quotes.toscrape.com/tag/love/,8.666299


# get_result(query, matriz_booleana) -> lista ordenada  

In [17]:
def get_result(query_raw: str, boolean_matrix: pd.DataFrame) -> list:
    #### hacer vector query
    corpus_stems = boolean_matrix.columns[1:]
    q = vector_consulta(query_raw, corpus_stems)
    #### calcular semejanza
    values = np.array(boolean_matrix.iloc[:,1:])
    semejanza_lista = get_sem_table(q, values)
    #### ordenar nombre y valores
    resultado = boolean_matrix[['name']].copy()
    resultado['semejanza'] = semejanza_lista
    ordenados = resultado.sort_values(by=['semejanza'], ascending=False)
    return list(ordenados['name'])

In [22]:
get_result("The Wizard of Menlo Park (now Edison, New Jersey) by a newspaper reporter", boolean_matrix)

['http://quotes.toscrape.com/author/Thomas-A-Edison/',
 'http://quotes.toscrape.com/',
 'http://quotes.toscrape.com/tag/paraphrased/page/1/',
 'http://quotes.toscrape.com/tag/obvious/page/1/',
 'http://quotes.toscrape.com/tag/misattributed-eleanor-roosevelt/page/1/',
 'http://quotes.toscrape.com/tag/simile/page/1/',
 'http://quotes.toscrape.com/tag/love/',
 'http://quotes.toscrape.com/page/2/',
 'http://quotes.toscrape.com/tag/failure/page/1/',
 'http://quotes.toscrape.com/tag/edison/page/1/',
 'http://quotes.toscrape.com/tag/value/page/1/',
 'http://quotes.toscrape.com/tag/love/page/1/',
 'http://quotes.toscrape.com/tag/success/page/1/',
 'http://quotes.toscrape.com/author/Eleanor-Roosevelt/',
 'http://quotes.toscrape.com/tag/simile/',
 'http://quotes.toscrape.com/author/Steve-Martin/',
 'http://quotes.toscrape.com/tag/truth/',
 'http://quotes.toscrape.com/tag/inspirational/',
 'http://quotes.toscrape.com/tag/friends/',
 'http://quotes.toscrape.com/tag/friendship/',
 'http://quotes.to

# PRUEBAS

In [46]:
# Define two NumPy arrays
array1 = np.array([0.5, 0.5, 0.5, 0.5, 0.5])
array2 = np.array([2, 3, 4, 5, 6])

# Perform element-wise multiplication
result = array1 * array2

# The 'result' array will contain the element-wise multiplication
print(result)


[1.  1.5 2.  2.5 3. ]


In [112]:
import numpy as np

# Define two matrices, each with rows representing vectors
q = np.array([1, 0, 1])
tabla = np.array([[1, 0, 1], [1, 1, 1], [0, 1, 1]])

# Perform element-wise multiplication between the two matrices
tabla_X_q = q * tabla

# The 'result' matrix will contain the element-wise multiplication
print(tabla_X_q)

[[1 0 1]
 [1 0 1]
 [0 0 1]]


In [136]:
pj = np.array([0.5, 0.5, 0.5])
qj = np.array([0.4, 0.5, 0.3])

In [137]:
1-qj

array([0.6, 0.5, 0.7])

In [138]:
pj*(1-qj)

array([0.3 , 0.25, 0.35])

In [139]:
(1-pj)

array([0.5, 0.5, 0.5])

In [140]:
qj*(1-pj)

array([0.2 , 0.25, 0.15])

In [141]:
(  pj*(1-qj)  )/ (  qj*(1-pj)  )

array([1.5       , 1.        , 2.33333333])

In [145]:
log_pj_qj_operation = np.log10((  pj*(1-qj)  )/ (  qj*(1-pj)  ))
log_pj_qj_operation

array([0.17609126, 0.        , 0.36797679])

In [152]:
tabla_bayes = tabla_X_q * log_pj_qj_operation
tabla_bayes

array([[0.17609126, 0.        , 0.36797679],
       [0.17609126, 0.        , 0.36797679],
       [0.        , 0.        , 0.36797679]])

In [155]:
semejanza = tabla_bayes.sum(axis=1)
semejanza

array([0.54406804, 0.54406804, 0.36797679])

In [33]:
( (q * boolean_matrix) *  np.log10((  pj*(1-qj)  )/ (  qj*(1-pj)  )) )

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/erick/tmp/crawling/roboto/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_235107/4066473308.py", line 1, in <module>
    ( (q * boolean_matrix) *  np.log10((  pj*(1-qj)  )/ (  qj*(1-pj)  )) )
       ~~^~~~~~~~~~~~~~~~
  File "/home/erick/tmp/crawling/roboto/venv/lib/python3.11/site-packages/pandas/core/generic.py", line 2102, in __array_ufunc__
    return arraylike.array_ufunc(self, ufunc, method, *inputs, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/erick/tmp/crawling/roboto/venv/lib/python3.11/site-packages/pandas/core/arraylike.py", line 273, in array_ufunc
    result = maybe_dispatch_ufunc_to_dunder_op(self, ufunc, method, *inputs, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "ops_dispatch.pyx", line 113, 